<a href="https://www.kaggle.com/hasanmahmood516/disaster-tweets-nlp?scriptVersionId=86430983" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/nlp-getting-started/sample_submission.csv
/kaggle/input/nlp-getting-started/train.csv
/kaggle/input/nlp-getting-started/test.csv


In [2]:
import spacy
from spacy.matcher import PhraseMatcher
nlp = spacy.load('en_core_web_sm')
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import Pipeline

from sklearn import metrics

In [3]:
train = pd.read_csv("/kaggle/input/nlp-getting-started/train.csv")
test = pd.read_csv("/kaggle/input/nlp-getting-started/test.csv")
sample = pd.read_csv("/kaggle/input/nlp-getting-started/sample_submission.csv")

In [4]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7613 entries, 0 to 7612
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        7613 non-null   int64 
 1   keyword   7552 non-null   object
 2   location  5080 non-null   object
 3   text      7613 non-null   object
 4   target    7613 non-null   int64 
dtypes: int64(2), object(3)
memory usage: 297.5+ KB


In [5]:
train.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [6]:
tfidf_vector = TfidfVectorizer()

In [7]:
#Let's just use the tweets for now and ignore the other information.

X = train['text'] # the features we want to analyze
y = train['target'] # the labels, or answers, we want to test against

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

classifier = LogisticRegression()

# Create pipeline using Tfidf
pipe = Pipeline([('vectorizer', tfidf_vector),
                 ('classifier', classifier)])

# model generation
pipe.fit(X_train,y_train)

# Predicting with a test dataset
predicted = pipe.predict(X_test)

# Model Accuracy
print("Logistic Regression Accuracy:",metrics.accuracy_score(y_test, predicted))
print("Logistic Regression Precision:",metrics.precision_score(y_test, predicted))
print("Logistic Regression Recall:",metrics.recall_score(y_test, predicted))

Logistic Regression Accuracy: 0.7977675640183848
Logistic Regression Precision: 0.7777777777777778
Logistic Regression Recall: 0.7022653721682848


In [8]:
#Now try to incorporate the keyword and location columns. To do this, we will first try vectorising each column separately
#using the tfidf vectoriser.

X2 = train[["keyword", "location", "text"]]
y2 = train["target"]

X2["keyword"] = X2["keyword"].fillna("")
X2["location"] = X2["location"].fillna("")

def tfidf_vectorizer(corpus):
    """
    Given a corpus input, returns the vectorization of the corpus using tfidf.
    """
    tfidf_corpus = TfidfVectorizer()
    corpus_vectors = tfidf_corpus.fit_transform(corpus)
    return corpus_vectors

key_vectors = tfidf_vectorizer(X2["keyword"].to_list())
location_vectors = tfidf_vectorizer(X2["location"].to_list())
text_vectors = tfidf_vectorizer(X2["text"].to_list())


X2["keyword"] = key_vectors.toarray()
X2["location"] = location_vectors.toarray()
X2["text"] = text_vectors.toarray()

X2_train, X2_test, y2_train, y2_test = train_test_split(X2, y2, test_size=0.2)

model2 = LogisticRegression()

model2.fit(X2_train,y2_train)

# Predicting with a test dataset
predicted2 = model2.predict(X2_test)

# Model Accuracy
print("Accuracy:",metrics.accuracy_score(y2_test, predicted2))
print("Precision:",metrics.precision_score(y2_test, predicted2))
print("Recall:",metrics.recall_score(y2_test, predicted2))


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pa

Accuracy: 0.5771503611293499
Precision: 0.6363636363636364
Recall: 0.010819165378670788


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [9]:
np.unique(predicted2)

array([0, 1])

In [10]:
key_vectors.toarray()

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [11]:
#Try keyword only, i.e. leave out location.

X3 = train[["keyword", "text"]]
y3 = train["target"]

X3["keyword"] = X3["keyword"].fillna("")

key_vectors3 = tfidf_vectorizer(X3["keyword"].to_list())
text_vectors3 = tfidf_vectorizer(X3["text"].to_list())


X3["keyword"] = key_vectors3.toarray()
X3["text"] = text_vectors3.toarray()

X3_train, X3_test, y3_train, y3_test = train_test_split(X3, y3, test_size=0.2)

model3 = LogisticRegression()

model3.fit(X3_train,y3_train)

# Predicting with a test dataset
predicted3 = model3.predict(X3_test)

# Model Accuracy
print("Accuracy:",metrics.accuracy_score(y3_test, predicted3))
print("Precision:",metrics.precision_score(y3_test, predicted3))
print("Recall:",metrics.recall_score(y3_test, predicted3))

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':


Accuracy: 0.5666447800393959
Precision: 0.875
Recall: 0.010510510510510511


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]


In [12]:
#Try location only, i.e. leave out keyword.

X4 = train[["location", "text"]]
y4 = train["target"]

X4["location"] = X4["location"].fillna("")

key_vectors4 = tfidf_vectorizer(X4["location"].to_list())
text_vectors4 = tfidf_vectorizer(X4["text"].to_list())


X4["location"] = key_vectors4.toarray()
X4["text"] = text_vectors4.toarray()

X4_train, X4_test, y4_train, y4_test = train_test_split(X4, y4, test_size=0.2)

model4 = LogisticRegression()

model4.fit(X4_train,y4_train)

# Predicting with a test dataset
predicted4 = model4.predict(X4_test)

# Model Accuracy
print("Accuracy:",metrics.accuracy_score(y4_test, predicted4))
print("Precision:",metrics.precision_score(y4_test, predicted4))
print("Recall:",metrics.recall_score(y4_test, predicted4))

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':


Accuracy: 0.5778069599474721
Precision: 0.0
Recall: 0.0


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]


In [13]:
from sklearn.linear_model import RidgeClassifier

In [14]:
#Try and use RidgeClassifier to see if it performs better.

X5 = train['text'] # the features we want to analyze
y5 = train['target'] # the labels, or answers, we want to test against

X5_train, X5_test, y5_train, y5_test = train_test_split(X5, y5, test_size=0.2)

model5 = RidgeClassifier()

tfidf_vector5 = TfidfVectorizer()

# Create pipeline using Tfidf
pipe5 = Pipeline([('vectorizer', tfidf_vector5),
                 ('classifier', model5)])

# model generation
pipe5.fit(X5_train,y5_train)

# Predicting with a test dataset
predicted5 = pipe5.predict(X5_test)

# Model Accuracy
print("Accuracy:",metrics.accuracy_score(y5_test, predicted5))
print("Precision:",metrics.precision_score(y5_test, predicted5))
print("Recall:",metrics.recall_score(y5_test, predicted5))

Accuracy: 0.8010505581089954
Precision: 0.7792642140468228
Recall: 0.7315541601255887


In [15]:
#The results show a slight improvement. Let's see if there is any difference when we include keyword and location.

In [16]:
#Incorporate the keyword and location columns with ridge classifier.

X6 = X2
y6 = train["target"]

X6_train, X6_test, y6_train, y6_test = train_test_split(X6, y6, test_size=0.2)

model6 = RidgeClassifier()

model6.fit(X6_train,y6_train)

# Predicting with a test dataset
predicted6 = model6.predict(X6_test)

# Model Accuracy
print("Accuracy:",metrics.accuracy_score(y6_test, predicted6))
print("Precision:",metrics.precision_score(y6_test, predicted6))
print("Recall:",metrics.recall_score(y6_test, predicted6))


Accuracy: 0.5896257386736704
Precision: 0.8333333333333334
Recall: 0.023547880690737835


In [17]:
#Again, we see a small improvement, but still significantly worse than when we only used the text.

In [18]:
from sklearn.feature_extraction.text import CountVectorizer

count_vectorizer = CountVectorizer()

In [19]:
#Use BoW vectorizer.

X7 = train['text'] # the features we want to analyze
y7 = train['target'] # the labels, or answers, we want to test against

X7_train, X7_test, y7_train, y7_test = train_test_split(X7, y7, test_size=0.2)

model7= RidgeClassifier()

# Create pipeline using Tfidf
pipe7= Pipeline([('vectorizer', count_vectorizer),
                 ('classifier', model7)])

# model generation
pipe7.fit(X7_train,y7_train)

# Predicting with a test dataset
predicted7 = pipe7.predict(X7_test)

# Model Accuracy
print("Accuracy:",metrics.accuracy_score(y7_test, predicted7))
print("Precision:",metrics.precision_score(y7_test, predicted7))
print("Recall:",metrics.recall_score(y7_test, predicted7))

Accuracy: 0.7800393959290873
Precision: 0.7903225806451613
Recall: 0.7050359712230215


In [20]:
#Keyword and location with BoW.

X8 = train[["keyword", "location", "text"]]
y8 = train["target"]

X8["keyword"] = X8["keyword"].fillna("")
X8["location"] = X8["location"].fillna("")

def BoW_vectorizer(corpus):
    """
    Given a corpus input, returns the vectorization of the corpus using BoW.
    """
    corpus_vectors = count_vectorizer.fit_transform(corpus)
    return corpus_vectors

key_vectors8 = BoW_vectorizer(X8["keyword"].to_list())
location_vectors8 = BoW_vectorizer(X8["location"].to_list())
text_vectors8 = BoW_vectorizer(X8["text"].to_list())


X8["keyword"] = key_vectors8.toarray()
X8["location"] = location_vectors8.toarray()
X8["text"] = text_vectors8.toarray()

X8_train, X8_test, y8_train, y8_test = train_test_split(X8, y8, test_size=0.2)

model8 = RidgeClassifier()

model8.fit(X8_train,y8_train)

# Predicting with a test dataset
predicted8 = model8.predict(X8_test)

# Model Accuracy
print("Accuracy:",metrics.accuracy_score(y8_test, predicted8))
print("Precision:",metrics.precision_score(y8_test, predicted8))
print("Recall:",metrics.recall_score(y8_test, predicted8))


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pa

Accuracy: 0.5581089954038083
Precision: 0.7058823529411765
Recall: 0.01764705882352941


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [21]:
#These are the best results we've had so far including keyword and location columns, but still not good. 

In [22]:
"""
How do we incorporate keyword and location columns effectively? 
Why do they make the results so much worse compared to just using the text? 
Perhaps there is an issue with so many rows having the same keyword/location? Maybe this information isn't actually helpful? 
Maybe we could encode them in a way which makes them useful, e.g. scale keyword encoding according to how often the keyword 
appears in the corresponding text? 
Would be good to look at correlation/ mutual information to see if the keyword/location columns are good features to use at all.

"""

"\nHow do we incorporate keyword and location columns effectively? \nWhy do they make the results so much worse compared to just using the text? \nPerhaps there is an issue with so many rows having the same keyword/location? Maybe this information isn't actually helpful? \nMaybe we could encode them in a way which makes them useful, e.g. scale keyword encoding according to how often the keyword \nappears in the corresponding text? \nWould be good to look at correlation/ mutual information to see if the keyword/location columns are good features to use at all.\n\n"

In [23]:
predict1 = pipe.predict(test["text"]) 

submit1 = sample.copy()
submit1["target"] = predict1

submit1.to_csv("sub1", index=False)

#score: 0.782721, rank at submission: 609/865